In [1]:
import pandas as pd
import tensorflow as tf

import autokeras as ak

from settings import INPUT_FILE, CORR_GROUP, OUTPUT_FILE
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import logging

logging.basicConfig(format='%(asctime)s %(message)s', filename='autokeras.log', level=logging.DEBUG)
logging.info('Started training')



2022-07-20 15:55:34.246011: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-20 15:55:34.246115: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)

def create_supervised_dataset(df, target, feats, n_in=1, n_out=1):
    cols, names = list(), list()
    n_vars = len(feats)
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[feats].shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[target].shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(1)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(1)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg.values

In [3]:
df = pd.read_csv(INPUT_FILE, index_col='ts')
#df = df.drop('Unnamed: 0', 1)
df.index = pd.to_datetime(df.index)

df_2 = df.loc[:,np.invert(unique_cols(df))]
# Average window
# df_2 = df.groupby(np.arange(len(df))//60).mean()

scaler = MinMaxScaler()
d = scaler.fit_transform(df_2)
scaled_df = pd.DataFrame(d, columns=df_2.columns, index=df_2.index)
results = []
train_split = int(0.7*scaled_df.shape[0])
val_split = int(0.9*scaled_df.shape[0])
predict_from = 1
predict_until = 1
lookback = 15

In [4]:
for k in CORR_GROUP:
    data = create_supervised_dataset(scaled_df, k, CORR_GROUP[k], n_in=lookback, n_out=1)
    data_train = data[:train_split]
    data_cv = data[train_split:val_split]
    
    data_x_train, data_y_train = data_train[:, :-1], data_train[:, -1]
    print(data_x_train.shape, data_y_train.shape)

    data_x_val, data_y_val = data_cv[:, :-1], data_cv[:, -1]
    print(data_x_val.shape, data_y_val.shape)
    
    data_test = data[val_split:int(1*scaled_df.shape[0])]
    data_x_test, data_y_test = data_test[:, :-1], data_test[:, -1]
    data_x_test = data_x_test.reshape(-1, lookback, len(CORR_GROUP[k]))

    clf = ak.TimeseriesForecaster(
        lookback=lookback,
        predict_from=predict_from,
        predict_until=predict_until,
        max_trials=3,
        project_name=f'autokeras_ml/{k}_forecaster',
        objective="mean_squared_error",
        overwrite=True
    )

    # Train the TimeSeriesForecaster with train data
    clf.fit(
        x=data_x_train,
        y=data_y_train,
        validation_data=(data_x_val, data_y_val),
        epochs=10,
        batch_size=64
    )

    model = clf.export_model()
    print(model.summary())
    model.save(f'models/{k}_autokeras.h5')
    # Evaluate the best model with testing data.
    print(model.evaluate(data_x_test, data_y_test)[0])
    break


Trial 3 Complete [00h 01m 53s]
mean_squared_error: 0.014870687387883663

Best mean_squared_error So Far: 0.010276983492076397
Total elapsed time: 00h 06m 03s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
433/433 [==============================] - 11s 21ms/step - loss: 0.0566 - mean_squared_error: 0.0566 - val_loss: 0.1838 - val_mean_squared_error: 0.1838
Epoch 2/10
433/433 [==============================] - 8s 19ms/step - loss: 0.0236 - mean_squared_error: 0.0236 - val_loss: 0.1354 - val_mean_squared_error: 0.1354
Epoch 3/10
433/433 [==============================] - 8s 19ms/step - loss: 0.0237 - mean_squared_error: 0.0237 - val_loss: 0.0864 - val_mean_squared_error: 0.0864
Epoch 4/10
433/433 [==============================] - 8s 19ms/step - loss: 0.0217 - mean_squared_error: 0.0217 - val_loss: 0.0481 - val_mean_squared_error: 0.0481
Epoch 5/10
433/433 [==============================] - 8s 19ms/step - loss: 0.0186 - mean_squared_error: 0.0186 - val_loss: 0.0384 - val_mean_squared_er

ValueError: in user code:

    File "/home/poliveira/Electricity-Forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1557, in test_function  *
        return step_function(self, iterator)
    File "/home/poliveira/Electricity-Forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1546, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/poliveira/Electricity-Forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1535, in run_step  **
        outputs = model.test_step(data)
    File "/home/poliveira/Electricity-Forecasting/lib/python3.8/site-packages/keras/engine/training.py", line 1499, in test_step
        y_pred = self(x, training=False)
    File "/home/poliveira/Electricity-Forecasting/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/poliveira/Electricity-Forecasting/

In [4]:
k='P_SUM'
train_split = int(0.7*scaled_df.shape[0])
val_split = int(0.9*scaled_df.shape[0])
data_train = scaled_df[:train_split]
data_cv = scaled_df[train_split:val_split]
data_x = data_train[CORR_GROUP[k]]
data_y = data_train[k]

data_x_val = data_cv[CORR_GROUP[k]]
data_y_val = data_cv[k]


In [5]:
for k in CORR_GROUP:
    data_test = create_supervised_dataset(scaled_df, k, CORR_GROUP[k], n_in=lookback, n_out=1)[val_split:]
    data_x_test, data_y_test = data_test[:, :-1], data_test[:, -1]
    data_x_test = data_x_test.reshape(-1, lookback, len(CORR_GROUP[k]))
    print(data_x_test.shape)
    print(data_y_test.shape)
    model = tf.keras.models.load_model(f'models/{k}_autokeras.h5')
    print(model.evaluate(data_x_test, data_y_test)[0])
    

(3941, 15, 19)
(3941,)


2022-07-20 15:55:46.812582: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-20 15:55:46.812678: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-20 15:55:46.812726: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (semantic): /proc/driver/nvidia/version does not exist
2022-07-20 15:55:46.813294: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


124/124 [==============================] - 4s 8ms/step - loss: 0.0208 - mean_squared_error: 0.0208
0.020842669531702995
(3941, 15, 6)
(3941,)
124/124 [==============================] - 3s 6ms/step - loss: 0.0010 - mean_squared_error: 0.0010  
0.0010008270619437099
(3941, 15, 1)
(3941,)
124/124 [==============================] - 1s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049
0.0048624672926962376
(3941, 15, 1)
(3941,)


KeyboardInterrupt: 